In [ ]:
import sys
sys.path.insert(0,'c:/MyDocs/integrated/') # adjust to your setup

%run "catalog_support.py" 
showHeader('Raw Disclosures',line2='Sixty days of FracFocus changes',use_remote=True)

In [ ]:
import datetime
today = datetime.datetime.today()
edate = today - datetime.timedelta(days=35)
print('earlist shown',edate)

In [ ]:
import os
import pickle
import openFF.common.text_handlers as th
arc_dir = r"D:\openFF_archive\diff_dicts"
# arc_dir = r"C:\MyDocs\integrated\archive\out"
# arc_dir = r"C:\MyDocs\integrated\openFF_archive\diff_dicts"
# arc_dir = r"\\M2\d\openFF_archive"
diff_fns = os.listdir(arc_dir)
added = []
changed = []
removed = []

for fn in diff_fns:
    tdate = datetime.datetime(int(fn[10:14]),int(fn[15:17]),int(fn[18:20]))
    # print(tdate)
    if tdate>= edate:
        with open(os.path.join(arc_dir,fn),'rb') as f:
            diff_dic = pickle.load(f)
        if len(diff_dic['removed_disc'])>0:
            t = diff_dic['removed_disc'].copy()
            # print(t.head())
            t['date_changed'] = tdate
            t['change_type'] = 'removed'
            removed.append(t)
        if len(diff_dic['added_disc'])>0:
            t = diff_dic['added_disc'].copy()
            t['date_changed'] = tdate
            t['change_type'] = 'added'
            added.append(t)
        if len(diff_dic['changed_disc'])>0:
            t = diff_dic['changed_disc'].copy()
            t['date_changed'] = tdate
            t['change_type'] = 'modified'            
            changed.append(t)
alllists = added + changed + removed
wholeset = pd.concat(alllists,sort=True)
# wholeset

In [ ]:
wholeset['job_end_date'] = wholeset.JobEndDate.str.split().str[0]
wholeset['job_end_date'] = pd.to_datetime(wholeset.job_end_date,format="%m/%d/%Y")
wholeset['FF_disc'] = wholeset.apply(lambda x: th.getFFLink(x), axis=1)
wholeset['TBWV'] = wholeset.TotalBaseWaterVolume.astype('float64')
wholeset['TBWV'] = wholeset.TBWV.map(lambda x: th.round_sig(x,5))

In [ ]:
wholeset[['FF_disc','job_end_date','StateName','CountyName','OperatorName','TBWV','change_type','date_changed']]

In [ ]:
wholeset.change_type.value_counts()

In [ ]:
t = wholeset[wholeset.change_type=='added'].date_changed.value_counts()
t = t.reset_index().sort_values('date_changed')
t.plot('date_changed','count',style='o')

In [ ]:
wholeset[wholeset.change_type=='changed'].date_changed.value_counts()